# What is happening at the heat of LLMs?

Before transformers, recurrent neural networks (RNNs) were considered the cutting edge in Natural Language Processing (NLP). An RNN is a type of neural network where outputs from previous steps are fed as inputs to the current step. This characteristic enables an RNN to retain information from previous steps, making them well-suited for sequential data like text. In the context of NLP, an RNN takes an input, such as a word or character, processes it through its network, and generates a vector known as the hidden state. If you are unfamiliar with RNNs, don't worry, you don't need to know the detailed workings of RNNs to follow this discussion. 

One area where RNNs played an important role was in the development of machine translation systems, where the model translates text from one language to another. However, the word sequence in one language might be different from another one due to the grammatical structures in the source and target language. To address this issue we can use an encoder-decoder architecture. The encoder's role is to convert input sequence information into a numerical representation, typically referred to as the final hidden state. The encoder updates its hidden state at each step, trying to capture the entire meaning of the input sentence in the final hidden state. The decoder then takes this final hidden state to start generating the translated sentence, one word at a time. \
However, a significant challenge of this architecture lies in the fact that the final hidden state of the encoder creates an information bottleneck. it has to represent the meaning of the whole input sequence because this is all the decoder has access to when generating the output. This is especially challenging for long sequences, where information at the start of the sequence might be lost in the process of compressing everything to a single, fixed representation.

To address this challenge, an "attention mechanism" is introduced, permitting the decoder to selectively access different hidden states of the encoder. But, why selective? Using all the states at the same time would create a huge input for the decoder, the attention mechanism lets the decoder assign a different amount of weight, or "attention" to each of the encoder states at every decoding timestep. \
Researchers, as detailed in the paper "Attention is all you need," have demonstrated that RNN architectures are not required for NLP applications such as machine translation and proposed a transformer architecture with a “self-attention mechanism”.  

The main idea behind the self-attention mechanism is that instead of using fixed embeddings for each token, we can use the whole sequence to compute a weighted average of each embedding. Given a sequence of token embeddings $ x_{1}, ..., x_{n} $ self-attention produces a sequence of new embeddings $ x_{1}^{'}, ..., x_{n}^{'} $ where each $ x_{i}^{'} $ is a linear combination of all the $ x_{j}^{'},  j=1...n $: 

$(1) \; x_{i} = \sum \limits _{j=1} ^{n} w_{ji}x_{j}$


There are several ways to implement a self-attention layer. The original implementation  introduced in the paper “” is called "Scaled Dot-Product Attention" 

$ (2) \; Attention(Q,K,V) =  softmax(\frac{QK^{T}}{\sqrt{d_{k}}})V$

In [ ]:
! pip install torch==2.0.1

In [105]:
import torch
from torch import nn

Putting these steps together, we will have the following function:

Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. 

$(3) \; MultiHead(Q,K,V) =  Concat(head_{1}, ..., head_{h})W^{O} $ 

where, 

$ head_{i} = Attention(QW_{i}^{Q},KW_{i}^{K},VW{i}^{V})$ and, \
$ W_{i}^{Q} \in \mathbb{R}^{d_{model}\times d_{k}}, W_{i}^{K} \in \mathbb{R}^{d_{model} \times d_{k}}, W_{i}^{V} \in \mathbb{R}^{d_{model} \times d_{v}} and W^{O} \in \mathbb{R}^{d_{hv} \times d_{model}} $ are weight matrices.

These three weight matrices are used to project the embedded input tokens, x(i), into query, key, and value vectors.

These matrices transform input data into queries, keys, and values, which are crucial components of the attention mechanism. As the model is exposed to more data during training, it adjusts these trainable weights

# What are Query, Key and Value ? 

In attention mechanisms, we use terms like "key," "query," and "value" which come from information retrieval and databases. They help us store, search, and get information efficiently.

Think of a "query" like a search term you put into a database. It's what the model is currently focusing on or trying to understand, like a word in a sentence. The query helps the model figure out how much attention to give to other parts of the input.

A "key" is like an index in a database used for searching. Each item in the input sequence, such as each word in a sentence, has a key. These keys are matched with the query to find relevant information.

The "value" in this context is similar to the value in a key-value pair in a database. It represents the actual content or representation of the input items. Once the model determines which keys (and thus which parts of the input) are most relevant to the query (the current focus item), it retrieves the corresponding values. 

With this introduction, let's code our very first attention mechanism. Imagine we have an embedding model that generates embeddings in a 5 dimentional embedding space. Assume that our embedding model has generated the following embedding vectors for our input sentence "Write your first Attention mechanism".  

Please note that embedding values in this example are totally random and dosen't express any information. 

In [106]:
import torch
# Write your first Attention mechanism
inputs = torch.tensor(
  [[0.172, 0.295, 0.618, 0.459, 0.818, 0.071], # Write 
   [0.265, 0.563, 0.718, 0.323, 0.126, 0.235], # your                                               
   [0.206, 0.333, 0.044, 0.862, 0.152, 0.594], # first    
   [0.300, 0.505, 0.727, 0.495, 0.898, 0.954], # Attention     
   [0.095, 0.809, 0.596, 0.110, 0.447, 0.418]] # mechanism   
)

In [107]:
print(inputs.shape)

torch.Size([5, 6])


First we should generate attention weight scores, which simply is the dot product of each embedding vector with other embedding vecotrs. Dot product is used as a similarity function. 

That is $ Attention Scores \in \mathbb{R}^{d_{t}\times d_{t}} $ \
Where $ d_{t} $ is the number of input tokens (i.e., words), here $ d_{t} = 5 $   

In [108]:
attn_scores = torch.matmul(inputs, inputs.T) 
print("attention scores are: ", attn_scores)

attention scores are:  tensor([[1.3834, 0.9234, 0.7230, 1.6794, 1.0691],
        [0.9234, 1.0781, 0.7108, 1.3830, 1.0987],
        [0.7230, 0.7108, 1.2742, 1.3918, 0.7262],
        [1.6794, 1.3830, 1.3918, 2.8351, 1.7250],
        [1.0691, 1.0987, 0.7262, 1.7250, 1.4054]])


Then, we normalize the attention scores using softmax function. The main goal behind the normalization is to obtain attention weights that sum up to 1. 

$ (4) \; \sigma(\mathbf{z})_{i} = \frac{e^{z_{i}}}{\sum_{j=1}^{K} e^{z_{j}}}, for \; i = 1 ,...,K \; and \; \mathbf{z} \in \mathbb{R}^{K} $

In [109]:
attn_weights = torch.softmax(attn_scores, dim=1)
print("attention weights are: ", attn_weights)
print("Sums of all rows are: ",attn_weights.sum(dim=1))

attention weights are:  tensor([[0.2368, 0.1495, 0.1224, 0.3184, 0.1730],
        [0.1739, 0.2030, 0.1406, 0.2753, 0.2072],
        [0.1497, 0.1479, 0.2598, 0.2923, 0.1502],
        [0.1489, 0.1107, 0.1117, 0.4729, 0.1558],
        [0.1649, 0.1698, 0.1170, 0.3176, 0.2307]])
Sums of all rows are:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [110]:
context_vectors = torch.matmul(attn_weights, inputs)  
print(context_vectors) # 5x6

tensor([[0.2175, 0.4955, 0.5936, 0.4391, 0.5944, 0.5007],
        [0.2149, 0.5191, 0.5831, 0.4257, 0.5291, 0.4928],
        [0.2204, 0.4831, 0.5122, 0.5017, 0.5102, 0.5414],
        [0.2346, 0.5083, 0.6131, 0.4516, 0.6470, 0.6192],
        [0.2147, 0.5302, 0.5974, 0.4140, 0.5624, 0.5206]])


As you can see, context vectors are the same size as our inputs. In other way, we simply modified the embeddings to reflect the attention to other tokens as well.

# Self Attention

The paper "Attention Is All You Need" introduces <em> Scaled Dot-Product Attention </em>. For instance, when scaling up the embedding dimension, which is typically greater than thousand for GPT-like LLMs, large dot products can result in very small gradients during backpropagation due to the softmax function applied to them. As dot products increase, the softmax function behaves more like a step function, resulting in gradients nearing zero. These small gradients can drastically slow down learning. We suspect that for large values of dk, the dot products grow large in magnitude, pushing the softmax function into regions where it has extremely small gradients. 

<center><figure><img src="imgs/scaled-dot-product.png" alt="drawing" width="300"/><figcaption>Fig. 1: Scaled Dot-Product Attention.</figcaption></figure></center>    

Let's see figure 1 in code. First, we write a function to implement scaled dot-product. 

In [111]:
from math import sqrt
def scaled_dot_product_attention(Q, K, V):
    dim_k = K.size(-1)
    attn_scores = torch.matmul(Q, K.T)
    attn_weights = torch.softmax(attn_scores / sqrt(dim_k),dim=-1)
    return torch.matmul(attn_weights,V)

In [112]:
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.W_q = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_k = nn.Linear(embed_dim, head_dim, bias=False)
        self.W_v = nn.Linear(embed_dim, head_dim, bias=False)
        
    def forward(self, x):
        keys = self.W_k(x)
        queries = self.W_q(x)
        values = self.W_v(x)        
  
        attention_outputs = scaled_dot_product_attention(queries, keys, values)
        return attention_outputs    

# Multi-head attention
As we saw, the self-attention mechanism employs three independent linear transformations on each embedding to produce the query, key, and value vectors. Each projection has  its own set of trainable parameters, so that the model, especially the self-attention layer, can attend to various semantic features within the sequence and  learn to produce "good" context vectors.		 	 	 		
			

Additionally, it would be advantageous to incorporate multiple sets of linear projections, referred to as  “attention head”. But why several attention heads? The reason is the softmax function of a single head focuses on one similarity aspect. By employing several heads, the model simultaneously focuses on multiple aspects of similarity.

<center><figure><img src="imgs/multi-head-attention.png" alt="drawing" width="350"/><figcaption>Fig. 2: Multi-head Attention.</figcaption></figure></center>    

Now that we have implemented self-attention mechanism, let's move forward and implement multi-head attention mechanism according to figure 2.

In [136]:
embed_size = inputs.shape[1]
num_heads = 2
head_dim = embed_size // num_heads
print(f"Embedding size is: {embed_size}, head dimension is: {head_dim} and number of heads are: {num_heads}")

Embedding size is: 6, head dimension is: 3 and number of heads are: 2


In [140]:
class MultiHeadAttention(nn.Module):
    def __init__(self, embed_size, num_heads, head_dim):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttention(embed_size, head_dim) for _ in range(num_heads)])
        self.output_linear = nn.Linear(embed_size, embed_size)
        
    def forward(self, inputs):
        x = torch.cat([head(inputs) for head in self.heads], dim=-1)
        x = self.output_linear(x)
        return x   

Please note that the final linear layer is used to produce a tensor of the same size as our input tensor (i.e., 5x6 in our example). Let's check it out!

In [141]:
mult_head_attn = MultiHeadAttention(embed_size,num_heads,head_dim)
attn_output = mult_head_attn(inputs)
print(attn_output.shape)

torch.Size([5, 6])


As you can see it generates an output tensor with the expected size.

If we look at the model architecture depicted in figure 3, there is a fully connected feed-forward network, which is applied to each position separately and identically. This consists of two linear transformations with a ReLU activation in between.

$(5) \; FFN(x) = max(0, xW_{1} + b_{1})W_{2} + b_{2} $

<center><figure><img src="imgs/transformer architecture.png" alt="drawing" width="400"/><figcaption>Fig. 3: Transformer architecture.</figcaption></figure></center> 

In [118]:
ff_hidden_size = 4 * embed_size 

In [129]:
class FeedForwardLayer(nn.Module):
    def __init__(self, embed_size, ff_hidden_size):
        super().__init__()
        self.x1 = nn.Linear(embed_size, ff_hidden_size)
        self.x2 = nn.Linear(ff_hidden_size, embed_size)
        self.relu = nn.ReLU()
    
    def forward(self,x):
        x = self.x1(x)
        x = self.relu(x)
        x = self.x2(x)
        return x
        

Moreover, a residual connection is employed around each of the two sub-layers, followed by layer normalization. That is, the output of each sub-layer is
$ LayerNorm(x + Sublayer(x)) $, where $ Sublayer(x) $ is the function implemented by the sub-layer itself [1].

Note: As a rule of thumb, the size of first layer is four times the size of the embeddings. 

In [132]:
feed_forward = FeedForwardLayer(embed_size, ff_hidden_size)
ff_out = feed_forward(inputs)
ff_out.shape

torch.Size([5, 6])

In [146]:
class Encoder(nn.Module):
    def __init__(self, embed_size, num_heads, head_dim, ff_hidden_size):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.multi_head_attn = MultiHeadAttention(embed_size,num_heads,head_dim)
        self.feed_forward = FeedForwardLayer(embed_size,ff_hidden_size)
        
    def forward(self,x):
        x = self.norm1(x + self.multi_head_attn(x))
        x = self.norm2( x + self.feed_forward(x))
        return x

Now, we can easily add the pieces together and build the encoder part as shown in the left side of figure 3. 

In [149]:
encoder = Encoder(embed_size,num_heads,head_dim,ff_hidden_size)
encoder_output = encoder(inputs)
encoder_output.shape

torch.Size([5, 6])

# Positional Encoding
Positional encoding augments the token embeddings by adding position-dependent information about the tokens in a sequence. To this end, we add "positional encodings" to the input embeddings at the bottoms of the encoder and decoder stacks. 

An easy solution could be to start token positions from 0 and continue untill all tokens are counted. However, this solution may end up with very large numbers in a large sequence of tokens. Instead, a positional encoding vector is created for each position, meaning a positional encoding matrix can be created to represent all the possible positions a word can take. 

There are many choices of positional encodings, the authors in [1] used the sine and cosine functions to generate a unique vector for each position in the sequence. 

$ PE(k, 2i) = sin(\frac {pos}{1000^{\frac{2i}{d_{embed}}}})  $

$ PE(k, 2i+1) = cos(\frac {pos}{1000^{\frac{2i+1}{d_{embed}}}})  $


# References: 

[1] Ashish Vaswan, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin. "Attention Is All You Need". 	arXiv:1706.03762v7. 